In [1]:
import urllib 
import math
from datetime import date
import re
import time
from IPython.display import clear_output
from datetime import date
import sys
import pandas as pd
import numpy as np
class DataGrapper:
    
    def updateTeams(self,from_year,to_year):
        df = pd.read_csv('teamId.csv',index_col=0)
        teamMatchUrl = 'http://app.gooooal.com/teamAllMatch.do?sid={}&tid={}&t=0&lang=tr'
        teams = df.values
        for i in range(teams.shape[0]):
            thisDf = None
            for year in range(from_year,to_year):
                url = teamMatchUrl.format(year,teams[i,0])
                print(url)
                tdf = pd.read_html(url,attrs={'id':'fb_match_odds_result_table_0_0'},encoding='UTF-8')[0]
                tdf = tdf.drop(df.index[[0]])
                if thisDf is None:
                    thisDf=tdf
                else:
                    thisDf =thisDf.append(tdf)
            print(teams[i,1])
            thisDf.to_csv("teams/"+teams[i,1]+".csv",index=False)

    def getOdds(self,date):
        dateStr = "{}-{}-{}".format(date.year,date.month,date.day)
        print("get {}".format(dateStr))
        url = 'http://info.livescore123.com/1x2/companyhistory.aspx?id=432&company=HK+Jockey+Club'
        data = {'matchdate': dateStr,'Submit': 'Search','key': ""}
        f = urllib.request.urlopen(
                url     = url,
                data    = urllib.parse.urlencode(data).encode('utf8')
            )
        raw= f.read()
        df = pd.read_html(raw,attrs={'class':'schedule'},header=0)[0]
        df = df[df['League'] == 'ENG PR'] 
        df = df [['Home','Away','HW','D','AW']]
        return df 
    def updateOdds(self,greater_than,origin, out_odds):
        footballdata = pd.read_csv(origin)
        footballdata['Date'] = pd.to_datetime(footballdata['Date'],dayfirst=True)
        if 'JocH' not in footballdata:
            footballdata['JocH']=pd.Series(np.zeros(shape=(footballdata.shape[0],)), index=footballdata.index) 
            footballdata['JocD'] = pd.Series(np.zeros(shape=(footballdata.shape[0],)), index=footballdata.index) 
            footballdata['JocA'] = pd.Series(np.zeros(shape=(footballdata.shape[0],)), index=footballdata.index)         
        dates = footballdata['Date'].drop_duplicates()
        dates  = dates[dates > greater_than]
        for indx, d in enumerate(dates):
            clear_output()
            print("{}/{} -- {}".format(indx,dates.shape[0],d))
            sys.stdout.flush()
            odds = None
            err_count =0
            while err_count < 5:
                try:
                    odds = self.getOdds(d)
                    break
                except Exception as e:
                    print (e)
                    time.sleep(3)
                err_count = err_count +1
            tempdf = footballdata[footballdata['Date'] == d]
            tempdf = tempdf.sort(columns='HomeTeam')
            odds = odds.sort(columns='Home')
            tempdf['JocH']= odds['HW'].values
            tempdf['JocD'] = odds['D'].values
            tempdf['JocA'] = odds['AW'].values
           # print(odds)
            footballdata.update(tempdf)
        footballdata.to_csv(out_odds,index=False)
    def readLastestMatches(self):
        url = 'http://bet.hkjc.com/football/odds/odds_had.aspx?ci=en-US'
        content= urllib.request.urlopen(url).read().decode('utf-8')
        #print(content)
        p2 = '<img src="/nas/jcbwinfo/football/info/images/.*?" alt="(.*?)"'
        p1= '<img src="/nas/jcbwinfo/football/info/images/.*?" alt="(.*?)".*?>'
        league = []
        for m in re.finditer(p1,content,flags=re.M):
            league.append(m.group(1))
        df= pd.read_html(url,attrs={'class':'tOdds'},header =0)[1]
        df = df.dropna(subset=['Expected StopSelling Time'])
        df['Unnamed: 1']=league
        ta = df['Teams(Home vs Away)'].values
        print(ta.shape)
        homeAways =[]
        for v in ta:
            homeAways.append([e.strip() for e in (v.split(' vs '))])
        nha = np.array(homeAways)
        df['HomeTeam']=nha[:,0]
        df['AwayTeam']=nha[:,1]
        rawTimes = df['Expected StopSelling Time'].values
        timesStr = [(v.split()[0])+'/2016' for v in rawTimes]
        print(timesStr)
        df['Date'] = pd.to_datetime(timesStr,dayfirst=True)
        df= df[df['Unnamed: 1']=='Eng Premier']
        df["JocH"] =df["Odds"].values
        df["JocD"]= df["Unnamed: 7"].values
        df["JocA"] = df["Unnamed: 8"].values
        df = df[["HomeTeam","AwayTeam","Date","JocH","JocD","JocA"]]
        return df.dropna(subset=['JocD'])
        #TODO: find epl name, -> epl only -> sort name -> return h,a,date,odds
        

In [18]:
tg = DataGrapper()
tg.updateOdds(date(2014,8,15),"dataSet/E2014.csv","dataSet/E2014odds.csv")

95/96 -- 2015-05-24 00:00:00
get 2015-5-24


In [23]:
df.to_csv("dataSet/future.csv",index=False)

In [2]:
from bs4 import BeautifulSoup
import datetime
import re
def getContentSoup (url):
        count = 0
        content=""
        while(count < 10):
            try:
                content = urllib.request.urlopen(url).read().decode('utf-8')
                break
            except:
                print("re-try")
                time.sleep(5)
                count = count+1
            if count ==10:
                raise Exception("error {}".format(url))
        soup = BeautifulSoup(content,'html.parser')
        return soup
def grapMatchInfo(url):

    from bs4 import BeautifulSoup
    import urllib 
    import math
    from datetime import date
    import datetime
    import re
    import time
    def getContentSoup (url):
        count = 0
        content=""
        while(count < 10):
            try:
                content = urllib.request.urlopen(url).read().decode('utf-8')
                break
            except:
                print("re-try")
                sys.stdout.flush()
                time.sleep(5)
                count = count+1
            if count ==10:
                raise Exception("error {}".format(url))
        soup = BeautifulSoup(content,'html.parser')
        return soup
    
    def summary():
        soup = getContentSoup(url)
        temp = soup.find("div","teams").text.split("\n")[1].split(',')[2:]
        tempstr = " ".join("".join(temp).strip().split(" ")[0:3])
        date = datetime.datetime.strptime(tempstr,"%B %d %Y")
        
        poss = [float(e.text[0:-1]) for e in soup.select("div#summary_possessions text")]
        #extract home id
        homeurl = soup.select("a.tabs-primary__tab-link")[1].get('href')
        homeId = -1 
        for m in re.finditer('/statszone/.*?/matches/.*?/team-stats/(\d+?)/OVERALL_01',homeurl,re.M):
            homeId = m.group(1)
        return date , poss,homeId
    def getTeamStat(homeId):
        
        #passing third
        atkthirdUrl = '/team-stats/{}/1_PASS_08'
        soup = getContentSoup(url+atkthirdUrl.format(homeId))
        awayurl=(soup.select("ul#statzone_pitch_team_filter li.last a")[0].get('href'))
        awayId = -1
        for m in re.finditer('/statszone/.*?/matches/.*?/team-stats/(\d+?)/1_PASS_08',awayurl,re.M):
            awayId = m.group(1)
        
        awayName = soup.select("ul#statzone_pitch_team_filter li.last a")[0].text
        homeName = soup.select("ul#statzone_pitch_team_filter li.first a")[0].text
        teamIds = [homeId,awayId]
        headers = ['atk_3rd','atk_3rd_tot','red_goal','red_on','red_tot','chance_goal', 'chance_tot',
                  'successful_tackle','tackle_tot','successful_clearance','clearance_tot','interceptions','blocks']
        headers = ['H_'+ t for t in headers] + ['A_'+t for t in headers]
        data = []
        for tid in teamIds:
            #name
            #attacking third
            soup = getContentSoup(url+atkthirdUrl.format(tid))
            atkThird = [ int(e.text) for e in soup.select("div.title-wrapper span")]
            data +=atkThird

            #red area
            redurl='/team-stats/{}/0_SHOT_055'.format(tid)
            soup= getContentSoup(url+redurl)
            matchs = ['stroke:yellow;stroke-width:3','stroke:blue;stroke-width:3']
            red_values = [0,0,0]
            for e in soup.select("line.pitch-object") :
                style =  e.get('style')
                red_values[2]+=1
                for indx, m in enumerate(matchs):
                    if style == m:
                        red_values[indx] +=1
                        break
            data+=red_values
            #chance created
            soup= getContentSoup(url+'/team-stats/{}/1_PASS_11'.format(tid))
            matchs = ['stroke:yellow;stroke-width:3']
            chance_values = [0,0]
            for e in soup.select("line.pitch-object") :
                style =  e.get('style')
                chance_values[1]+=1
                for indx, m in enumerate(matchs):
                    if style == m:
                        chance_values[indx] +=1
                        break
            data+=chance_values
            
            #def
            soup= getContentSoup(url+'/team-stats/{}/OVERALL_04'.format(tid))
            def_headers = ['successful_tackle','failed_tackle','successful_clearance','failed_clearance','interceptions','blocks']
            def_values = [0 for i in range(len(def_headers))]
            for e in soup.select("image.pitch-object"):
                href =e.get("xlink:href")
                for indx, header  in enumerate(def_headers):
                    if header in href:
                        def_values[indx]+=1
                        break

            def_values[1]+=def_values[0]
            def_values[3]+= def_values[2]
            data+=def_values
      
        
        
        return homeName, awayName, data,headers
        
    date,poss,homeId = summary()
    homeName, awayName, data,headers=  getTeamStat(homeId)
    n_data = np.array(data).reshape(2,len(headers)/2).T
    print(n_data)
    n_headers = np.ravel(np.array(headers).reshape(2,len(headers)/2).T)
    n_headers = np.hstack(["Date","HomeTeam","AwayTeam","H_poss","A_poss",n_headers])
    result = np.hstack([date,np.ravel(np.vstack([[homeName,awayName],poss,n_data]))])

    return result,n_headers 
    
    


In [3]:
result,n_headers  = grapMatchInfo('http://www.fourfourtwo.com/statszone/8-2015/matches/803479')

[30, 18, 22, 7, 22, 6]
[77, 116, 1, 3, 9, 1, 10, 30, 48, 22, 29, 22, 6]
[19, 5, 22, 12, 20, 6]
[77, 116, 1, 3, 9, 1, 10, 30, 48, 22, 29, 22, 6, 89, 146, 1, 1, 8, 1, 14, 19, 24, 22, 34, 20, 6]
[[ 77  89]
 [116 146]
 [  1   1]
 [  3   1]
 [  9   8]
 [  1   1]
 [ 10  14]
 [ 30  19]
 [ 48  24]
 [ 22  22]
 [ 29  34]
 [ 22  20]
 [  6   6]]


/home/y/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:131: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/y/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:133: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [4]:
print(result)
print(n_headers)
pd.DataFrame(result.reshape(1,result.shape[0]),columns=n_headers)

[datetime.datetime(2016, 4, 2, 0, 0) 'Liverpool' 'Tottenham Hotspur' '42.2'
 '57.8' '77' '89' '116' '146' '1' '1' '3' '1' '9' '8' '1' '1' '10' '14'
 '30' '19' '48' '24' '22' '22' '29' '34' '22' '20' '6' '6']
['Date' 'HomeTeam' 'AwayTeam' 'H_poss' 'A_poss' 'H_atk_3rd' 'A_atk_3rd'
 'H_atk_3rd_tot' 'A_atk_3rd_tot' 'H_red_goal' 'A_red_goal' 'H_red_on'
 'A_red_on' 'H_red_tot' 'A_red_tot' 'H_chance_goal' 'A_chance_goal'
 'H_chance_tot' 'A_chance_tot' 'H_successful_tackle' 'A_successful_tackle'
 'H_tackle_tot' 'A_tackle_tot' 'H_successful_clearance'
 'A_successful_clearance' 'H_clearance_tot' 'A_clearance_tot'
 'H_interceptions' 'A_interceptions' 'H_blocks' 'A_blocks']


,Date,HomeTeam,AwayTeam,H_poss,A_poss,H_atk_3rd,A_atk_3rd,H_atk_3rd_tot,A_atk_3rd_tot,H_red_goal,...,H_tackle_tot,A_tackle_tot,H_successful_clearance,A_successful_clearance,H_clearance_tot,A_clearance_tot,H_interceptions,A_interceptions,H_blocks,A_blocks
0,2016-04-02 00:00:00,Liverpool,Tottenham Hotspur,42.2,57.8,77,89,116,146,1,...,48,24,22,22,29,34,22,20,6,6


In [6]:
soup= getContentSoup('http://www.fourfourtwo.com/statszone/8-2015/matches/803479/team-stats/6/OVERALL_04')
def_headers = ['successful_tackle','failed_tackle','successful_clearance','failed_clearance','interceptions','blocks']
def_values = [0 for i in range(len(def_headers))]
for e in soup.select("image.pitch-object"):
    href =e.get("xlink:href")
    print(href)
    for indx, header  in enumerate(def_headers):
        if header in href:
            def_values[indx]+=1
            break

#def_values[1]+=def_values[0]
#def_values[3] += def_values[2]



/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_tackle.png
/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_tackle.png
/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_tackle.png
/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_tackle.png
/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_tackle.png
/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_tackle.png
/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_tackle.png
/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_tackle.png
/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_tackle.png
/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_tackle.png
/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_tackle.png
/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_tackle.png
/sites/fourfourtwo.com/modul

In [7]:
def_values

[19, 5, 22, 12, 20, 6]

In [30]:
soup= getContentSoup('http://www.fourfourtwo.com/statszone/8-2015/matches/803479/team-stats/14/1_PASS_11')
matchs = ['stroke:yellow;stroke-width:3','stroke:deepskyblue;stroke-width:3']
values = [0,0,0]
for e in soup.select("line.pitch-object") :
    style =  e.get('style')
    print(style)
    values[2]+=1
    for indx, m in enumerate(matchs):
        if style == m:
            values[indx] +=1
            break
            
            
        



stroke:deepskyblue;stroke-width:3
stroke:deepskyblue;stroke-width:3
stroke:deepskyblue;stroke-width:3
stroke:deepskyblue;stroke-width:3
stroke:yellow;stroke-width:3
stroke:deepskyblue;stroke-width:3
stroke:deepskyblue;stroke-width:3
stroke:deepskyblue;stroke-width:3
stroke:deepskyblue;stroke-width:3
stroke:deepskyblue;stroke-width:3


In [31]:
values

[1, 9, 10]

In [47]:
import os, http.cookiejar, urllib.request
from cookielib import Cookie, CookieJar

def getReferee():
    cj = CookieJar()
    # Cookie(version, name, value, port, port_specified, domain, 
    # domain_specified, domain_initial_dot, path, path_specified, 
    # secure, discard, comment, comment_url, rest)
    c = Cookie(None, 'asdf', None, '80', '80', 'www.foo.bar', 
           None, None, '/', None, False, False, 'TestCookie', None, None, None)
    cj.set_cookie(c)
    print (cj)
    url = 'http://www.premierleague.com/content/premierleague/en-gb/referees/appointments.html?paramSearchContext=MATCH_WEEK&paramMatchWeek=31'
    return
    req = urllib.request.Request(url)
    req.add_header('Accept-Language','en-US;q=0.6,en;q=0.4')
    content = urllib.request.urlopen(req).read().decode('utf-8')
    print(content)
    #soup= getContentSoup(url)
    #print(soup.prettify())
    dfs = pd.read_html(url)
    dfs[0]["Date"] =datetime.date(2016,4,2)
    dfs[1]["Date"] = datetime.date(2016,4,3)
    print(dfs)



ImportError: No module named 'cookielib'

In [48]:
getReferee()










<!--[if lt IE 7]>      <html lang="zh" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="zh" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="zh" class="no-js lt-ie9"> <![endif]-->
<!--[if IE 9]>         <html lang="zh" class="no-js lt-ie10"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="zh"> <!--<![endif]-->

    
 

<head>
    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1">
    <meta name="apple-mobile-web-app-capable" content="yes">
    <meta name="apple-mobile-web-app-status-bar-style" content="black-translucent">
    <meta name="HandheldFriendly" content="true"> 
    <meta name="MobileOptimized" content="320">
    <meta name="description" content="">
    <meta name="author" content="">
    




 


    <meta property="wb:webmaster" con

ValueError: No tables found

In [6]:
from IPython.parallel import Client
import time
from IPython.display import clear_output
import sys
def getYearMatch(year):
    
    listurl ='http://www.fourfourtwo.com/statszone/results/8-{}'.format(year)
    baseurl = 'http://www.fourfourtwo.com'
    soup = getContentSoup(listurl)
    tot_links = [e.get('href') for e in  (soup.select("td.link-to-match a"))]
    dates = []
    h_names = []
    a_names =[]
    posses = []
    h_atks = []
    a_atks=[]
    h_ins=[]
    a_ins=[]
    def workerFunc(args):
        import sys
        links =args[0]
        grapMatchInfo = args[1]
        print("start working")
        dates = []
        h_names = []
        a_names =[]
        posses = []
        h_atks = []
        a_atks=[]
        h_ins=[]
        a_ins=[]
        for indx, link in enumerate(links):
            print("task: {}/{}".format(indx,len(links)))
            sys.stdout.flush()
            try:
                date ,homeName, awayName ,poss, h_atkThird, a_atkThird, h_interception,a_interception = grapMatchInfo(baseurl + link)
                print("{} {} {}".format(date,homeName,awayName))
                dates.append(date)
                h_names.append(homeName)
                a_names.append(awayName)
                posses.append(poss)
                h_atks.append(h_atkThird)
                a_atks.append(a_atkThird)
                h_ins.append(h_interception)
                a_ins.append(a_interception)
            except Exception as e:
                print(e)
                print("error")
                break
            
        print("finish")
        return [dates,h_names,a_names,posses,h_atks,a_atks,h_ins,a_ins]
    def finishCallback(returnArgs):
        print("complete")
        dates = []
        h_names = []
        a_names =[]
        posses = []
        h_atks = []
        a_atks=[]
        h_ins=[]
        a_ins=[]
        for args in returnArgs:
            dates=dates+args[0]
            h_names = h_names + args[1]
            a_names = a_names + args[2]
            posses = posses+args[3]
            h_atks=h_atks + args[4]
            a_atks = a_atks + args[5]
            h_ins = h_ins + args[6]
            a_ins = a_ins + args[7]
        return dates,h_names,a_names,posses,h_atks,a_atks,h_ins,a_ins
        
    n_work = 4
    pool = Client()
    print(pool.ids)
    each_n = int(len(tot_links)/n_work)
    works = [tot_links[i*each_n:(i+1)*each_n] for i in range(n_work)]
    works[0].append(tot_links[4*each_n:])
    results = []
    for i in range(n_work):
        results.append(pool[i].apply_async(f=workerFunc,args=[works[i],grapMatchInfo,getContentSoup]))
    def wait_watching_stdout(ars, dt=5, truncate=500):
        while True:
            allReady = True 
            printed = False
            for ar in ars:
                if not ar.ready():
                    allReady = False
                    stdouts = ar.metadata['stdout']
                    if not any(stdouts):
                        continue
                    if printed == False:
                        clear_output()
                        printed = True
                    # clear_output doesn't do much in terminal environments
                    print ('-' * 30)
                    print ("%.3fs elapsed" % ar.elapsed)
                    print ("")
                    print(ar._targets)
                    #print ("[ stdout %2i ]" % (ar._targets))
                    print(ar.metadata['stdout'][-truncate:])
                    sys.stdout.flush()
            if allReady == True:
                return
            time.sleep(dt)      
    wait_watching_stdout(results)
    returnArgs = [ e.get() for e in results]
    pool.close()
    return finishCallback(returnArgs)

    
    

/home/y/anaconda3/lib/python3.5/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


In [4]:
from IPython.parallel import Client
import time
from IPython.display import clear_output
import sys
def getYearMatch(year):
    
    listurl ='http://www.fourfourtwo.com/statszone/results/8-{}'.format(year)
    baseurl = 'http://www.fourfourtwo.com'
    soup = getContentSoup(listurl)
    tot_links = [e.get('href') for e in  (soup.select("td.link-to-match a"))]
    results =[]
    n_header=None
    print("start working")
    for indx, link in enumerate(tot_links):
        clear_output()
        print("task: {}/{}".format(indx,len(tot_links)))
        sys.stdout.flush()
        try:
            result,n_headers = grapMatchInfo(baseurl + link)
            print("{} {} {}".format(result[0],result[1],result[2]))
            results.append(result)
            n_header = n_headers
        except Exception as e:
            print(e)
            print("error")
            break
            
        print("finish")
    
 
    return results,n_header

In [5]:
results,n_header= getYearMatch(2015)

task: 310/311
[[121  75]
 [174 110]
 [  0   2]
 [  2   4]
 [  6  12]
 [  0   0]
 [  8  12]
 [ 16  22]
 [ 29  36]
 [ 10  23]
 [ 17  33]
 [ 13  12]
 [  5   2]]
2015-08-08 00:00:00 Chelsea Swansea City
finish


[array([datetime.datetime(2016, 4, 2, 0, 0), 'Aston Villa', 'Chelsea',
        '41.4', '58.6', '101', '83', '136', '119', '0', '4', '1', '2', '6',
        '7', '0', '2', '11', '6', '24', '19', '32', '31', '9', '14', '15',
        '23', '21', '8', '0', '4'], dtype=object),
 array([datetime.datetime(2016, 4, 2, 0, 0), 'West Ham United',
        'Crystal Palace', '66.4', '33.6', '75', '41', '118', '81', '1', '2',
        '1', '2', '9', '10', '0', '0', '13', '6', '12', '18', '21', '34',
        '25', '17', '34', '23', '9', '14', '3', '6'], dtype=object),
 array([datetime.datetime(2016, 4, 2, 0, 0), 'Bournemouth',
        'Manchester City', '45.8', '54.2', '77', '145', '110', '187', '0',
        '4', '1', '2', '6', '10', '0', '3', '6', '13', '18', '26', '32',
        '38', '15', '10', '20', '16', '17', '15', '0', '3'], dtype=object),
 array([datetime.datetime(2016, 4, 2, 0, 0), 'Norwich City',
        'Newcastle United', '41.3', '58.7', '81', '93', '132', '131', '3',
        '2', '2', '2', 

In [160]:
def getTempMatch():
    
    tot_links = ['/statszone/8-2015/matches/803167',
                 '/statszone/8-2015/matches/803170',
                 '/statszone/8-2015/matches/803168']
                 
    baseurl = 'http://www.fourfourtwo.com'
    dates = []
    h_names = []
    a_names =[]
    posses = []
    h_atks = []
    a_atks=[]
    h_ins=[]
    a_ins=[]
    print("start working")
    for indx, link in enumerate(tot_links):
        clear_output()
        print("task: {}/{}".format(indx,len(tot_links)))
        sys.stdout.flush()
        try:
            date ,homeName, awayName ,poss, h_atkThird, a_atkThird, h_interception,a_interception = grapMatchInfo(baseurl + link)
            print("{} {} {}".format(date,homeName,awayName))
            dates.append(date)
            h_names.append(homeName)
            a_names.append(awayName)
            posses.append(poss)
            h_atks.append(h_atkThird)
            a_atks.append(a_atkThird)
            h_ins.append(h_interception)
            a_ins.append(a_interception)
        except Exception as e:
            print(e)
            print("error")
            break
            
        print("finish")
    
 
    return dates,h_names,a_names,posses,h_atks,a_atks,h_ins,a_ins

In [8]:

df = pd.DataFrame(results, columns=n_header)

In [9]:
df.sort(columns="Date",ascending=False)

/home/y/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Date,HomeTeam,AwayTeam,H_poss,A_poss,H_atk_3rd,A_atk_3rd,H_atk_3rd_tot,A_atk_3rd_tot,H_red_goal,...,H_tackle_tot,A_tackle_tot,H_successful_clearance,A_successful_clearance,H_clearance_tot,A_clearance_tot,H_interceptions,A_interceptions,H_blocks,A_blocks
0,2016-04-02,Aston Villa,Chelsea,41.4,58.6,101,83,136,119,0,...,32,31,9,14,15,23,21,8,0,4
5,2016-04-02,Arsenal,Watford,69.1,30.9,238,56,284,99,4,...,17,27,21,20,30,27,25,19,2,3
7,2016-04-02,Liverpool,Tottenham Hotspur,42.2,57.8,77,89,116,146,1,...,48,24,22,22,29,34,22,20,6,6
6,2016-04-02,Sunderland,West Bromwich Albion,56.5,43.5,101,51,159,105,0,...,24,24,37,23,48,36,14,19,1,6
1,2016-04-02,West Ham United,Crystal Palace,66.4,33.6,75,41,118,81,1,...,21,34,25,17,34,23,9,14,3,6
4,2016-04-02,Stoke City,Swansea City,47.2,52.8,75,86,125,125,1,...,24,19,18,18,26,26,18,16,4,2
3,2016-04-02,Norwich City,Newcastle United,41.3,58.7,81,93,132,131,3,...,16,17,20,22,31,33,5,12,1,2
2,2016-04-02,Bournemouth,Manchester City,45.8,54.2,77,145,110,187,0,...,32,38,15,10,20,16,17,15,0,3
8,2016-03-20,Southampton,Liverpool,49.2,50.8,114,111,163,164,2,...,32,20,23,21,33,26,22,12,4,5
9,2016-03-20,Newcastle United,Sunderland,59.9,40.1,74,72,116,124,1,...,35,40,25,23,31,38,9,6,3,3


In [10]:
df.to_csv('dataSet/match2015.csv',index=False)